한 연구에 따르면 데이터 분석의 80%는 데이터를 정리하고 적절한 형태로 변형하는 것 입니다. 이 장에서는 데이터셋을 원하는 형태로
변형할 수 있도록 새로운 판다스 기술을 소개합니다 먼저 큰 데이터셋을 간결한 피벗 테이블로 요약하는 방법을 살펴봅니다. 그런 다음
집계된 데이터셋을 집계된 데이터셋을 분할하는 방법을 알아보면서 반대로 변환하는 방법도 공부합니다.

# 8.1 넓은 데이터와 좁은 데이터

# 8.2 DataFrame에서 피벗 테이블 생성

In [1]:
import pandas as pd

In [2]:
pd.read_csv('sales_by_employee.csv').head()

,Date,Name,Customer,Revenue,Expenses
0,1/1/20,Oscar,Logistics XYZ,5250,531
1,1/1/20,Oscar,Money Corp.,4406,661
2,1/2/20,Oscar,PaperMaven,8661,1401
3,1/3/20,Oscar,PaperGenius,7075,906
4,1/4/20,Oscar,Paper Pound,2524,1767


In [4]:
sales = pd.read_csv('sales_by_employee.csv',
                   parse_dates = ['Date'])
sales.tail()

,Date,Name,Customer,Revenue,Expenses
21,2020-01-01,Creed,Money Corp.,4430,548
22,2020-01-02,Creed,Average Paper Co.,8026,1906
23,2020-01-02,Creed,Average Paper Co.,5188,1768
24,2020-01-04,Creed,PaperMaven,3144,1314
25,2020-01-05,Creed,Money Corp.,938,1053


## 8.2.1 pivot_table 메서드

In [5]:
sales.pivot_table(index = 'Date')

,Expenses,Revenue
Date,,
2020-01-01,637.500000,4293.500000
2020-01-02,1244.400000,7303.000000
2020-01-03,1313.666667,4865.833333
2020-01-04,1450.600000,3948.000000
2020-01-05,1196.250000,4834.750000


In [6]:
# 다음 두 줄의 코드는 결과가 동일합니다
sales.pivot_table(index = 'Date')
sales.pivot_table(index = 'Date', aggfunc = 'mean')

,Expenses,Revenue
Date,,
2020-01-01,637.500000,4293.500000
2020-01-02,1244.400000,7303.000000
2020-01-03,1313.666667,4865.833333
2020-01-04,1450.600000,3948.000000
2020-01-05,1196.250000,4834.750000


In [7]:
sales.pivot_table(index = 'Date', aggfunc = 'sum')

,Expenses,Revenue
Date,,
2020-01-01,3825,25761
2020-01-02,6222,36515
2020-01-03,7882,29195
2020-01-04,7253,19740
2020-01-05,4785,19339


In [8]:
sales.pivot_table(
    index = 'Date', values = 'Revenue', aggfunc = 'sum'
)

,Revenue
Date,
2020-01-01,25761
2020-01-02,36515
2020-01-03,29195
2020-01-04,19740
2020-01-05,19339


In [9]:
sales.pivot_table(
    index = 'Date',
    columns = 'Name',
    values = 'Revenue',
    aggfunc = 'sum'
)

Name,Creed,Dwight,Jim,Michael,Oscar
Date,,,,,
2020-01-01,4430.0,2639.0,1864.0,7172.0,9656.0
2020-01-02,13214.0,NaN,8278.0,6362.0,8661.0
2020-01-03,NaN,11912.0,4226.0,5982.0,7075.0
2020-01-04,3144.0,NaN,6155.0,7917.0,2524.0
2020-01-05,938.0,7771.0,NaN,7837.0,2793.0


In [11]:
sales.pivot_table(
    index = 'Date',
    columns = 'Name',
    values = 'Revenue',
    aggfunc = 'sum',
    fill_value = 0
)

Name,Creed,Dwight,Jim,Michael,Oscar
Date,,,,,
2020-01-01,4430,2639,1864,7172,9656
2020-01-02,13214,0,8278,6362,8661
2020-01-03,0,11912,4226,5982,7075
2020-01-04,3144,0,6155,7917,2524
2020-01-05,938,7771,0,7837,2793


In [12]:
sales.pivot_table(
    index = 'Date',
    columns = 'Name',
    values = 'Revenue',
    aggfunc = 'sum',
    fill_value = 0,
    margins = True
)

Name,Creed,Dwight,Jim,Michael,Oscar,All
Date,,,,,,
2020-01-01 00:00:00,4430,2639,1864,7172,9656,25761
2020-01-02 00:00:00,13214,0,8278,6362,8661,36515
2020-01-03 00:00:00,0,11912,4226,5982,7075,29195
2020-01-04 00:00:00,3144,0,6155,7917,2524,19740
2020-01-05 00:00:00,938,7771,0,7837,2793,19339
All,21726,22322,20523,35270,30709,130550


In [14]:
sales.pivot_table(
    index = 'Date',
    columns = 'Name',
    values = 'Revenue',
    aggfunc = 'sum',
    fill_value = 0,
    margins = True,
    margins_name = 'Total'
)

Name,Creed,Dwight,Jim,Michael,Oscar,Total
Date,,,,,,
2020-01-01 00:00:00,4430,2639,1864,7172,9656,25761
2020-01-02 00:00:00,13214,0,8278,6362,8661,36515
2020-01-03 00:00:00,0,11912,4226,5982,7075,29195
2020-01-04 00:00:00,3144,0,6155,7917,2524,19740
2020-01-05 00:00:00,938,7771,0,7837,2793,19339
Total,21726,22322,20523,35270,30709,130550


## 8.2.2 피벗 테이블의 추가 기능

In [15]:
sales.pivot_table(
    index = 'Date',
    columns = 'Name',
    values = 'Revenue',
    aggfunc = 'count'
)

Name,Creed,Dwight,Jim,Michael,Oscar
Date,,,,,
2020-01-01,1.0,1.0,1.0,1.0,2.0
2020-01-02,2.0,NaN,1.0,1.0,1.0
2020-01-03,NaN,3.0,1.0,1.0,1.0
2020-01-04,1.0,NaN,2.0,1.0,1.0
2020-01-05,1.0,1.0,NaN,1.0,1.0


In [17]:
sales.pivot_table(
    index = 'Date',
    columns = 'Name',
    values = 'Revenue',
    aggfunc = ['sum', 'count'],
    fill_value = 0
)

sum                            count                         
Name        Creed Dwight   Jim Michael Oscar Creed Dwight Jim Michael Oscar
Date                                                                       
2020-01-01   4430   2639  1864    7172  9656     1      1   1       1     2
2020-01-02  13214      0  8278    6362  8661     2      0   1       1     1
2020-01-03      0  11912  4226    5982  7075     0      3   1       1     1
2020-01-04   3144      0  6155    7917  2524     1      0   2       1     1
2020-01-05    938   7771     0    7837  2793     1      1   0       1     1

In [20]:
sales.pivot_table(
    index = 'Date',
    columns = 'Name',
    values = ['Revenue', 'Expenses'],
    fill_value = 0,
    aggfunc = {'Revenue':'min', 'Expenses':'min'}
)

Expenses                            Revenue                       \
Name          Creed Dwight   Jim Michael Oscar   Creed Dwight   Jim Michael   
Date                                                                          
2020-01-01      548    368  1305     412   531    4430   2639  1864    7172   
2020-01-02     1768      0   462     685  1401    5188      0  8278    6362   
2020-01-03        0    758  1923    1772   906       0   2703  4226    5982   
2020-01-04     1314      0   426    1857  1767    3144      0  2287    7917   
2020-01-05     1053   1475     0    1633   624     938   7771     0    7837   

                  
Name       Oscar  
Date              
2020-01-01  4406  
2020-01-02  8661  
2020-01-03  7075  
2020-01-04  2524  
2020-01-05  2793

In [21]:
sales.pivot_table(
    index = ['Name', 'Date'], values = 'Revenue', aggfunc = 'sum'
).head(10)

Revenue
Name   Date               
Creed  2020-01-01     4430
       2020-01-02    13214
       2020-01-04     3144
       2020-01-05      938
Dwight 2020-01-01     2639
       2020-01-03    11912
       2020-01-05     7771
Jim    2020-01-01     1864
       2020-01-02     8278
       2020-01-03     4226

In [22]:
sales.pivot_table(
    index = ['Date', 'Name'], values = 'Revenue', aggfunc = 'sum'
).head(10)

Revenue
Date       Name            
2020-01-01 Creed       4430
           Dwight      2639
           Jim         1864
           Michael     7172
           Oscar       9656
2020-01-02 Creed      13214
           Jim         8278
           Michael     6362
           Oscar       8661
2020-01-03 Dwight     11912

## 8.3 인덱스 레벨 스택과 언스택

In [23]:
sales.head()

,Date,Name,Customer,Revenue,Expenses
0,2020-01-01,Oscar,Logistics XYZ,5250,531
1,2020-01-01,Oscar,Money Corp.,4406,661
2,2020-01-02,Oscar,PaperMaven,8661,1401
3,2020-01-03,Oscar,PaperGenius,7075,906
4,2020-01-04,Oscar,Paper Pound,2524,1767


In [26]:
by_name_and_date = sales.pivot_table(
    index = 'Name',
    columns = 'Date',
    values = 'Revenue',
    aggfunc = 'sum'
)

by_name_and_date.head(2)

Date,2020-01-01,2020-01-02,2020-01-03,2020-01-04,2020-01-05
Name,,,,,
Creed,4430.0,13214.0,NaN,3144.0,938.0
Dwight,2639.0,NaN,11912.0,NaN,7771.0


In [27]:
by_name_and_date.stack().head(7)

Name    Date      
Creed   2020-01-01     4430.0
        2020-01-02    13214.0
        2020-01-04     3144.0
        2020-01-05      938.0
Dwight  2020-01-01     2639.0
        2020-01-03    11912.0
        2020-01-05     7771.0
dtype: float64

In [29]:
sales_by_customer = sales.pivot_table(
    index = ['Customer', 'Name'],
    values = 'Revenue',
    aggfunc = 'sum'
)

sales_by_customer.head()

Revenue
Customer          Name            
Average Paper Co. Creed      13214
                  Jim         2287
Best Paper Co.    Dwight      2703
                  Michael    15754
Logistics XYZ     Dwight      9209

In [30]:
sales_by_customer.unstack()

Revenue                                 
Name                 Creed  Dwight     Jim  Michael   Oscar
Customer                                                   
Average Paper Co.  13214.0     NaN  2287.0      NaN     NaN
Best Paper Co.         NaN  2703.0     NaN  15754.0     NaN
Logistics XYZ          NaN  9209.0     NaN   7172.0  5250.0
Money Corp.         5368.0     NaN  8278.0      NaN  4406.0
Paper Pound            NaN  7771.0  4226.0      NaN  5317.0
PaperGenius            NaN  2639.0  1864.0  12344.0  7075.0
PaperMaven          3144.0     NaN  3868.0      NaN  8661.0

## 8.4 데이터셋 피벗 해제

In [31]:
sales.head(1)

,Date,Name,Customer,Revenue,Expenses
0,2020-01-01,Oscar,Logistics XYZ,5250,531


In [32]:
video_game_sales = pd.read_csv('video_game_sales.csv')
video_game_sales.head()

,Name,NA,EU,JP,Other
0,Wii Sports,41.49,29.02,3.77,8.46
1,Super Mario Bros.,29.08,3.58,6.81,0.77
2,Mario Kart Wii,15.85,12.88,3.79,3.31
3,Wii Sports Resort,15.75,11.01,3.28,2.96
4,Pokemon Red/Pokemon Blue,11.27,8.89,10.22,1.00


In [33]:
video_game_sales.head(1)

,Name,NA,EU,JP,Other
0,Wii Sports,41.49,29.02,3.77,8.46


In [34]:
video_game_sales.melt(id_vars = 'Name', value_vars = 'NA').head()

,Name,variable,value
0,Wii Sports,NA,41.49
1,Super Mario Bros.,NA,29.08
2,Mario Kart Wii,NA,15.85
3,Wii Sports Resort,NA,15.75
4,Pokemon Red/Pokemon Blue,NA,11.27


In [35]:
regional_sales_columns = ['NA', 'EU', 'JP', 'Other']

video_game_sales.melt(
    id_vars = 'Name', value_vars = regional_sales_columns
)

,Name,variable,value
0,Wii Sports,NA,41.49
1,Super Mario Bros.,NA,29.08
2,Mario Kart Wii,NA,15.85
3,Wii Sports Resort,NA,15.75
4,Pokemon Red/Pokemon Blue,NA,11.27
...,...,...,...
66259,Woody Woodpecker in Crazy Castle 5,Other,0.00
66260,Men in Black II: Alien Escape,Other,0.00
66261,SCORE International Baja 1000: The Official Game,Other,0.00
66262,Know How 2,Other,0.00


In [41]:
video_game_sales_by_region = video_game_sales.melt(
    id_vars = "Name", 
    value_vars = regional_sales_columns,
    var_name = "Region",
    value_name = "Sales"
)

video_game_sales_by_region.head()

,Name,Region,Sales
0,Wii Sports,NA,41.49
1,Super Mario Bros.,NA,29.08
2,Mario Kart Wii,NA,15.85
3,Wii Sports Resort,NA,15.75
4,Pokemon Red/Pokemon Blue,NA,11.27


In [42]:
video_game_sales_by_region.pivot_table(
    index = 'Name', values = 'Sales', aggfunc = 'sum'
).head()

,Sales
Name,
'98 Koshien,0.40
.hack//G.U. Vol.1//Rebirth,0.17
.hack//G.U. Vol.2//Reminisce,0.23
.hack//G.U. Vol.3//Redemption,0.17
.hack//Infection Part 1,1.26


## 8.5 값의 목록 확장

In [43]:
recipes = pd.read_csv('recipes.csv')
recipes

,Recipe,Ingredients
0,Cashew Crusted Chicken,"Apricot preserves, Dijon mustard, curry powder..."
1,Tomato Basil Salmon,"Salmon filets, basil, tomato, olive oil, Parme..."
2,Parmesan Cheese Chicken,"Bread crumbs, Parmesan cheese, Italian seasoni..."


In [44]:
recipes['Ingredients'].str.split(',')

0    [Apricot preserves,  Dijon mustard,  curry pow...
1    [Salmon filets,  basil,  tomato,  olive oil,  ...
2    [Bread crumbs,  Parmesan cheese,  Italian seas...
Name: Ingredients, dtype: object

In [45]:
recipes['Ingredients'] = recipes['Ingredients'].str.split(',')
recipes

,Recipe,Ingredients
0,Cashew Crusted Chicken,"[Apricot preserves, Dijon mustard, curry pow..."
1,Tomato Basil Salmon,"[Salmon filets, basil, tomato, olive oil, ..."
2,Parmesan Cheese Chicken,"[Bread crumbs, Parmesan cheese, Italian seas..."


In [46]:
recipes.explode('Ingredients')

,Recipe,Ingredients
0,Cashew Crusted Chicken,Apricot preserves
0,Cashew Crusted Chicken,Dijon mustard
0,Cashew Crusted Chicken,curry powder
0,Cashew Crusted Chicken,chicken breasts
0,Cashew Crusted Chicken,cashews
1,Tomato Basil Salmon,Salmon filets
1,Tomato Basil Salmon,basil
1,Tomato Basil Salmon,tomato
1,Tomato Basil Salmon,olive oil
1,Tomato Basil Salmon,Parmesan cheese
